Notebook explores  MaterialsProject API: Determining stable phases from predicted energy values, calculating energies.

In [29]:
from mp_api.client import MPRester as New_MPRester
from urllib.parse import urlencode
import httplib2
from pymatgen.ext.matproj import MPRester as Legacy_MPRester
from emmet.core.thermo import ThermoType
from pymatgen.entries.mixing_scheme import MaterialsProjectDFTMixingScheme
from pymatgen.entries.computed_entries import ComputedEntry
from pymatgen.analysis.phase_diagram import PhaseDiagram
from pymatgen.core import Element

import os
import json
import time
import numpy as np

In [30]:
New_MP_api_key ="SECRETKEY"
Legacy_MP_api_key = "SECRETKEY"

data_dir = 'data'
enthalpies_file = data_dir + "\\fusion_enthalpies.json"
melt_temps_file = data_dir + "\\fusion_temperatures.json"

if not os.path.exists(enthalpies_file):
    enthalpies = {}
else:
    with open(enthalpies_file, "r") as file:
        enthalpies = json.load(file)

if not os.path.exists(melt_temps_file):
    melt_temps = {}
else:
    with open(melt_temps_file, "r") as file:
        melt_temps = json.load(file)

In [31]:
# returns the DFT convex hull of a given system with specified functionals
def get_dft_convexhull(components, verbose=False):
    dft_type = "GGA/GGA+U"
    # if dft_type not in dft_types:
    #     print("invalid DFT type")
    #     exit(1)
    if 'Yb' in components:
        dft_type = "GGA"

    if verbose:
        print("using DFT entries solved with", dft_type, "functionals")
    sys = '-'.join(sorted(components))

    # dft_type_path = '_'.join(dft_type.split('/'))
    # dft_entries_file = os.path.join(f"{data_dir}\\{sys}", f"{sys}_MP_ENTRIES_{dft_type_path}.json")
    dft_entries_file = os.path.join(f"{data_dir}\\{sys}", f"{sys}_ENTRIES_MP_GGA.json")

    if os.path.exists(dft_entries_file):
        with open(dft_entries_file, "r") as f:
            dft_entries = json.load(f)

        try:
            pd = PhaseDiagram(elements=[Element(c) for c in components],
                              entries=[ComputedEntry.from_dict(e) for e in dft_entries])
            if verbose:
                print(len(pd.stable_entries) - 2, "stable line compound(s) on the DFT convex hull\n")
            return pd
        except ValueError as e:
            print(f"error loading DFT entries from cache: {e}")

    # no cache or invalid cached data
    entries = []

    # using legacy MP energies (GGA)
    if dft_type == "GGA":
        with Legacy_MPRester(Legacy_MP_api_key) as MPR:
            entries = MPR.get_entries_in_chemsys(components, inc_structure=True)

    # using new MP energies (GGA/GGA+U, R2SCAN, GGA/GGA+U/R2SCAN)
    else:
        with New_MPRester(New_MP_api_key) as MPR:
            # if dft_type == "R2SCAN" or dft_type == "GGA/GGA+U/R2SCAN":
            #     scan_entries = MPR.get_entries_in_chemsys(components,
            #                                               additional_criteria={
            #                                                   'thermo_types': [ThermoType.R2SCAN]})
            if dft_type == "GGA/GGA+U" or dft_type == "GGA/GGA+U/R2SCAN":
                gga_entries = MPR.get_entries_in_chemsys(components,
                                                         additional_criteria={
                                                             'thermo_types': [ThermoType.GGA_GGA_U]})

        # if dft_type == "GGA/GGA+U/R2SCAN":
        #     entries = MaterialsProjectDFTMixingScheme().process_entries(scan_entries + gga_entries,
        #                                                                 verbose=verbose)
        if dft_type == "GGA/GGA+U":
            entries = MaterialsProjectDFTMixingScheme().process_entries(gga_entries, verbose=verbose)
        # elif dft_type == "R2SCAN":
        #     entries = MaterialsProjectDFTMixingScheme().process_entries(scan_entries, verbose=verbose)

    if verbose:
        print(f"caching DFT entry data as {dft_entries_file}...")
    dft_entries = [e.as_dict() for e in entries]
    for e in dft_entries:
        e.pop('structure')
        e.pop('data')
    with open(dft_entries_file, "w") as f:
        json.dump(dft_entries, f)

    try:
        pd = PhaseDiagram(elements=[Element(c) for c in components],
                          entries=[ComputedEntry.from_dict(e) for e in dft_entries])
        if verbose:
            print(len(pd.stable_entries) - 2, "stable line compound(s) on the DFT convex hull\n")
        return pd
    except ValueError as e:
        print(f"error with DFT entries downloaded from API: {e}")
        return None





In [33]:
get_dft_convexhull(['Cr','O'])

Retrieving ThermoDoc documents: 100%|██████████| 117/117 [00:00<00:00, 115303.94it/s]
c:\Users\andri\anaconda3\envs\matprojenv\lib\site-packages\pymatgen\entries\compatibility.py:1039: UserWarning: Failed to guess oxidation states for Entry mp-1206821-GGA+U (Cr2O). Assigning anion correction to only the most electronegative atom.
  warnings.warn(
c:\Users\andri\anaconda3\envs\matprojenv\lib\site-packages\pymatgen\entries\compatibility.py:1039: UserWarning: Failed to guess oxidation states for Entry mp-714896-GGA+U (Cr3O). Assigning anion correction to only the most electronegative atom.
  warnings.warn(
c:\Users\andri\anaconda3\envs\matprojenv\lib\site-packages\pymatgen\entries\compatibility.py:1039: UserWarning: Failed to guess oxidation states for Entry mp-20609-GGA+U (Cr3O). Assigning anion correction to only the most electronegative atom.
  warnings.warn(
c:\Users\andri\anaconda3\envs\matprojenv\lib\site-packages\pymatgen\entries\compatibility.py:1039: UserWarning: Failed to guess 

Cr-O phase diagram
5 stable phases: 
O2, CrO2, Cr2O3, Cr5O12, Cr

In [ ]:
components=["Fe","O"]
dft_type = "GGA/GGA+U"

ch = dm.get_dft_convexhull(components)
phases = []

# initialize phases from DFT entries on the hull
for entry in ch.stable_entries:
    try:
        composition = entry.composition.fractional_composition.as_dict()[components[1]]
    except KeyError:
        composition = 0

    # convert eV/atom to J/mol (96,485 J/mol per 1 eV/atom)
    phase = {'name': entry.name, 'comp': composition, 'points': [],
             'energy': 96485 * ch.get_form_energy_per_atom(entry)}
    phases.append(phase)

NameError: name 'dm' is not defined